In [2]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [4]:
study='interplay_13'


In [28]:
rp=dcm.read_file('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/BAKER/dicom/RP.201610113.AILEDENEZD_TT.dcm')
#rd=dcm.read_file('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/BAKER/dicom/RD.201610113.Dose_AILEDENEZDTE1.dcm')
#rd

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20180726'
(0008, 0013) Instance Creation Time              TM: '161835.341000'
(0008, 0016) SOP Class UID                       UI: RT Dose Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.7.950586737223.695174.20170125150117
(0008, 0020) Study Date                          DA: '20170111'
(0008, 0030) Study Time                          TM: '114828'
(0008, 0050) Accession Number                    SH: '103093944'
(0008, 0060) Modality                            CS: 'RTDOSE'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'CHIRA^CIPRIAN'
(0008, 1010) Station Name                        SH: 'SRVVARIAN-DB'
(0008, 1030) Study Description                   LO: 'TDM therapie SPC'
(0008, 103e) Series Description                  LO: 'Doses Eclipse'
(0008, 1048) Physi

# Create an mhd file from DICOM RT dose 

In [16]:
#get pixel array
rdpix=rd.pixel_array.astype(np.float32)

#Convert rt dose value into Gy value 
f=float(rd.DoseGridScaling)
rdpix1=rdpix*f 

#create an sitk image from this array
rdim=sitk.GetImageFromArray(rdpix1)

#Set origin of image from rt dose value
origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)
#Set spacing of image from rt dose value (x et y)
spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))
#for z direction
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)

#write rd image in '.raw'
sitk.WriteImage(rdim, '/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/results/emc_'+study+'.mhd')
#sitk.WriteImage(rdim, '/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/DEAK/results/emc_DEAK_oreille.mhd')

print(spacing)


print('emc_'+study+'.mhd --> ok')

[2.5, 2.5, 2.5]
emc_BAKER.mhd --> ok


# GATE output and RT dose DICOM

In [17]:
#Get the rd raw created  and gate image
refim=sitk.ReadImage('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/results/emc_'+study+'.mhd')
gateim=sitk.ReadImage('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/results/'+study+'/media/'+study+'.mhd')

#refim=sitk.ReadImage('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/DEAK/results/emc_DEAK_oreille.mhd')
#gateim=sitk.ReadImage('/media/sf_Documents/phd/work/gateTrueBeamModel/tb/fullCreator/results/DEAK_oreille/media/DEAK_oreille.mhd')

# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)                                          
ct_viewer(refpix)
ct_viewer(gatepix)

interactive(children=(IntSlider(value=50, description='myslice'), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=56, description='myslice', max=112), Output()), _dom_classes=('widget-in…

In [18]:
# compute true value gate output offset in the DICOM frame
ct_im=sitk.ReadImage('/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/images/'+study+'.mhd')

half_ct_im=[]
half_gateim=[]
void_actor=[]
new_origin=[]

for size, space in zip(ct_im.GetSize(), ct_im.GetSpacing()): half_ct_im.append((size*space)/2)
for size, space in zip(gateim.GetSize(), gateim.GetSpacing()): half_gateim.append((size*space)/2)

for vala, valb  in zip(half_gateim, half_ct_im): void_actor.append(vala-valb)
    
ct_im_origin=[ct_im.GetOrigin()[0], ct_im.GetOrigin()[1]*-1, ct_im.GetOrigin()[2]*-1]

for val1, val2 in zip(ct_im_origin, void_actor): new_origin.append(val1-val2)
    
gateim.SetOrigin(new_origin)
print('New Origin : ', gateim.GetOrigin())

New Origin :  (-145.85949999999997, -136.09387999999998, -202.5)


In [19]:
#to mask GATE images an get just BODY DOSE
#Resample (Downsampling) rd image into gate image size an resolution
refim_res=sitk.Resample(refim, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
refim_respix=sitk.GetArrayFromImage(refim_res)
ct_viewer(refim_respix)

interactive(children=(IntSlider(value=56, description='myslice', max=112), Output()), _dom_classes=('widget-in…

In [20]:
#to mask GATE images an get just BODY DOSE
#Resample (RESIZE) GATE image into EMC image size an resolution
gateim_res=sitk.Resample(gateim, refim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
gateim_respix=sitk.GetArrayFromImage(gateim_res)
ct_viewer(gateim_respix)

interactive(children=(IntSlider(value=50, description='myslice'), Output()), _dom_classes=('widget-interact',)…

In [21]:
#Create a boolean mask to apply on Gate image (remove dose outside the patient according to RTdose)
mask=np.ma.make_mask(refim_respix)
#apply on gate pixel array
gatepix_mask=gatepix*mask
ct_viewer(gatepix_mask)

interactive(children=(IntSlider(value=56, description='myslice', max=112), Output()), _dom_classes=('widget-in…

# Absolute dose conversion 

In [180]:
# ************ F0 ************
energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy

if energy == 6:
    GVref=0.000108726
    Npart=1033128667.0
    DrG=0.000108726
    
if energy == 9:
    GVref=0.000140945
    Npart=1136273663.0
    DrG=0.000140945

if energy == 12:
    GVref=0.000150559
    Npart=1086793583.0
    DrG=0.000150559
    
print(energy)

6


In [182]:
# ************ F1 ************

energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy

if energy == 6:
    GVref=0.00011708
    Npart=1033128667.0
    DrG=0.00011708
    
if energy == 9:
    GVref=0.00014613
    Npart=1136273663.0
    DrG=0.00014613
    
if energy == 12:
    GVref=0.00016055
    Npart=1086793583.0
    DrG=0.00016055

if energy == 18:
    GVref=0.00014757
    Npart=1086793583.0
    DrG=0.00014757

print(energy)

6


In [22]:
# ************ F2 ************

energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy

if energy == 6:
    GVref=0.00011727
    Npart=1033128667.0
    DrG=0.00011727
    
if energy == 9:
    GVref=0.00014731
    Npart=1136273663.0
    DrG=0.00014731
    
if energy == 12:
    GVref=0.00016086
    Npart=1086793583.0
    DrG=0.00016086
    
if energy == 18:
    GVref=0.00014752
    Npart=1086793583.0
    DrG=0.00014752

print(energy)

6


In [23]:
#Absolute dose conversion LUC SIMON
um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)
print('um:', um)
print('frac:', nFrac)
D_fac=(1.0/GVref)*0.01*um*nFrac

print (D_fac)

gatepix_abs=gatepix_mask*(1.0/GVref)*0.01*um*nFrac
print(gatepix_abs.max())

um: 345.0
frac: 16.0
470708.62113072403
59.180172


In [24]:
#Absolute dose conversion BIBLIO
GperPart=DrG/Npart

Dref=0.01
F=Dref/GperPart

um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

gatepix_abs=(gatepix_mask/Npart)*F*um*nFrac
print(gatepix_abs.max())

59.180176


In [25]:
print(0.95*gatepix_abs.max())
print(0.80*gatepix_abs.max())
print(0.60*gatepix_abs.max())
print(0.50*gatepix_abs.max())
print(0.25*gatepix_abs.max())
print(0.15*gatepix_abs.max())

56.2211669921875
47.344140625
35.508105468749996
29.590087890625
14.7950439453125
8.877026367187499


In [26]:
print(0.95*rdpix1.max())
print(0.80*rdpix1.max())
print(0.60*rdpix1.max())
print(0.50*rdpix1.max())
print(0.25*rdpix1.max())
print(0.15*rdpix1.max())



48.39137153625488
40.75062866210938
30.56297149658203
25.46914291381836
12.73457145690918
7.6407428741455075


In [97]:
#Create a raw image of gate with mask and a raw image from resampled rt dose dicom 
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(refim_res, '/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/results/emc_resampled_'+study+'.mhd')
sitk.WriteImage(gate_dose, '/home/gate/Documents/phd/work/gateTrueBeamModel/tb/fullCreator/patient/'+study+'/results/gate_F2_'+study+'.mhd')